# Newsletter Development Notebook

This notebook provides a complete 1:1 correspondence with the FastAPI newsletter processing system for rapid development and testing.

## Setup Instructions

### Environment Configuration
1. **Copy the environment template**: `cp .env.example .env.local`
2. **Configure your credentials** in `.env.local`:
   ```bash
   # LLM Configuration
   OPENAI_API_KEY=sk-your-openai-api-key-here
   
   # Neo4j Configuration  
   NEO4J_URI=bolt://localhost:7687
   NEO4J_USER=neo4j
   NEO4J_PASSWORD=your-neo4j-password
   
   # Processing Configuration
   ENTITY_CONFIDENCE_THRESHOLD=0.7
   FACT_CONFIDENCE_THRESHOLD=0.8
   ```

### Required Dependencies
```bash
# Core dependencies
pip install -r requirements.txt

# Notebook development dependencies
pip install -r requirements-notebook.txt
```

### Neo4j Setup
```bash
# Start Neo4j for development
./scripts/start-neo4j.sh

# Access Neo4j Browser: http://localhost:7474
# Username: neo4j, Password: your-neo4j-password
```

## Architecture Overview

This notebook mirrors the complete FastAPI application structure:

| **Notebook Cell** | **FastAPI File** | **Purpose** |
|-------------------|------------------|-------------|
| Cell 1 | `src/config.py` | Environment configuration |
| Cell 2 | `src/models/newsletter.py` | Data models |
| Cell 3 | `src/processors/html_processor.py` | HTML processing |
| Cell 4 | `src/processors/entity_extractor.py` | Entity extraction |
| Cell 5 | `src/graph/neo4j_client.py` | Neo4j operations |
| Cell 6 | `src/workflows/newsletter_processor.py` | Complete workflow |
| Cell 7 | Testing & Validation | Pipeline testing |
| Cell 8 | Code Export Helper | Sync back to FastAPI |

## Security Notes

- **Never commit credentials**: The notebook uses `.env.local` which is gitignored
- **Environment variables**: All sensitive data is loaded from environment variables
- **Local development**: This setup is designed for local development only

## Quick Start

1. Set up environment variables as described above
2. Run cells 1-5 to initialize all components
3. Run cell 7 to test the complete pipeline
4. Use cell 8 to export code back to FastAPI files

## Entity Types

The system extracts 6 types of entities:
- **Organization**: Companies, institutions, government bodies
- **Person**: Individuals mentioned in content
- **Product**: Software, hardware, services, models
- **Event**: Conferences, announcements, launches
- **Location**: Geographic locations
- **Topic**: Subject areas, technologies, fields of study

# Environment Setup and Configuration

**FastAPI File**: `/Users/paulbonneville/Developer/arrgh-fastapi/src/config.py`

**Purpose**: Environment configuration and dependency management for the notebook.

**What this mirrors**: The complete configuration system from the FastAPI app, including:
- `get_settings()` - Load configuration from environment variables
- `print_configuration_summary()` - Display current configuration
- `validate_configuration()` - Validate configuration completeness
- Environment variable management with .env.local for development
- Fallback configuration class for when FastAPI imports are unavailable

---

In [86]:
# Cell 1: Environment Setup and Configuration
import os
import sys
from pathlib import Path
from typing import List, Dict, Any, Optional
from datetime import datetime
import json

# Add src directory to path for imports
current_dir = Path.cwd()
project_root = current_dir.parent if current_dir.name == 'notebooks' else current_dir
src_path = project_root / 'src'
sys.path.insert(0, str(src_path))

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv(project_root / ".env.local")  # Use .env.local for development

# Import the proper configuration system from FastAPI
try:
    from config import get_settings, print_configuration_summary, validate_configuration
    
    # Get configuration using the FastAPI system
    config = get_settings()
    
    # Print configuration summary
    print_configuration_summary(config)
    
    # Validate configuration (but filter out Neo4j warning if it's actually working)
    validation_messages = validate_configuration(config)
    
    # Test Neo4j connection to verify if the password actually works
    neo4j_working = False
    if config.neo4j_password:
        try:
            from neo4j import GraphDatabase
            driver = GraphDatabase.driver(config.neo4j_uri, auth=(config.neo4j_user, config.neo4j_password))
            with driver.session() as session:
                session.run("RETURN 1 as test")
            driver.close()
            neo4j_working = True
        except:
            pass
    
    # Filter out Neo4j password warning if connection works
    if neo4j_working:
        validation_messages = [msg for msg in validation_messages if "Neo4j password" not in msg]
    
    if validation_messages:
        print("\n⚠️  Configuration Issues:")
        for message in validation_messages:
            print(f"  {message}")
    else:
        print("\n✅ Configuration is valid")
        
    print("\n✅ Using FastAPI configuration system")
    
except ImportError as e:
    print(f"⚠️ Could not import FastAPI config system: {e}")
    print("  Falling back to simple config class...")
    
    # Fallback configuration class
    class Config:
        """Configuration class mirroring src/config.py Settings."""
        
        # LLM Configuration
        openai_api_key = os.getenv("OPENAI_API_KEY")
        llm_model = os.getenv("LLM_MODEL", "gpt-4-turbo")
        llm_temperature = float(os.getenv("LLM_TEMPERATURE", "0.1"))
        llm_max_tokens = int(os.getenv("LLM_MAX_TOKENS", "2000"))
        
        # Neo4j Configuration
        neo4j_uri = os.getenv("NEO4J_URI", "bolt://localhost:7687")
        neo4j_user = os.getenv("NEO4J_USER", "neo4j")
        neo4j_password = os.getenv("NEO4J_PASSWORD")
        neo4j_database = os.getenv("NEO4J_DATABASE", "neo4j")
        
        # Processing Configuration
        max_entities_per_newsletter = int(os.getenv("MAX_ENTITIES_PER_NEWSLETTER", "100"))
        fact_extraction_batch_size = int(os.getenv("FACT_EXTRACTION_BATCH_SIZE", "10"))
        processing_timeout = int(os.getenv("PROCESSING_TIMEOUT", "300"))
        entity_confidence_threshold = float(os.getenv("ENTITY_CONFIDENCE_THRESHOLD", "0.7"))
        fact_confidence_threshold = float(os.getenv("FACT_CONFIDENCE_THRESHOLD", "0.8"))
        
        # Feature Flags
        enable_debug_mode = os.getenv("ENABLE_DEBUG_MODE", "false").lower() == "true"
        enable_async_processing = os.getenv("ENABLE_ASYNC_PROCESSING", "false").lower() == "true"
        
        # Security Configuration
        secret_key = os.getenv("SECRET_KEY")
    
    config = Config()
    neo4j_working = False
    
    # Test Neo4j connection
    if config.neo4j_password:
        try:
            from neo4j import GraphDatabase
            driver = GraphDatabase.driver(config.neo4j_uri, auth=(config.neo4j_user, config.neo4j_password))
            with driver.session() as session:
                session.run("RETURN 1 as test")
            driver.close()
            neo4j_working = True
        except:
            pass
    
    # Basic validation
    validation_messages = []
    if not config.openai_api_key or config.openai_api_key.startswith("sk-your-"):
        validation_messages.append("⚠️ OpenAI API key not properly configured")
    if not neo4j_working:
        validation_messages.append("⚠️ Neo4j connection failed - check password and service")
    
    print("🔧 Configuration loaded from environment:")
    print(f"  Environment file: .env.local")
    print(f"  LLM Model: {config.llm_model}")
    print(f"  Neo4j URI: {config.neo4j_uri}")
    print(f"  Max Entities: {config.max_entities_per_newsletter}")
    print(f"  Entity Confidence Threshold: {config.entity_confidence_threshold}")
    print(f"  Debug Mode: {config.enable_debug_mode}")
    
    if validation_messages:
        print("\n📋 Configuration Messages:")
        for message in validation_messages:
            print(f"  {message}")

# Environment validation
print("\n🔍 Environment Status:")
print(f"  Project root: {project_root}")
print(f"  Environment: {os.getenv('ENVIRONMENT', 'local')}")
print(f"  Python path includes src: {str(src_path) in sys.path}")

# Critical settings check
if not config.openai_api_key or config.openai_api_key.startswith("sk-your-"):
    print("\n❌ SETUP REQUIRED: Copy .env.example to .env.local and add your OpenAI API key")
    print("   cp .env.example .env.local")
    print("   # Edit .env.local with your actual API key")
else:
    print(f"\n✅ OpenAI API key configured (starts with: {config.openai_api_key[:7]}...)")

if neo4j_working:
    print(f"✅ Neo4j configured and connected: {config.neo4j_uri}")
else:
    print("❌ SETUP REQUIRED: Configure Neo4j password in .env.local")
    print("   # Run: ./scripts/start-neo4j.sh")
    print("   # Then add NEO4J_PASSWORD to .env.local")

# Import statements with fallback handling
print("\n📦 Loading dependencies...")
try:
    from bs4 import BeautifulSoup
    import html2text
    print("✅ HTML processing libraries loaded")
except ImportError as e:
    print(f"⚠️ HTML processing libraries not available: {e}")
    print("  Run: pip install beautifulsoup4 html2text")

try:
    from openai import OpenAI
    print("✅ OpenAI library loaded")
except ImportError as e:
    print(f"⚠️ OpenAI library not available: {e}")
    print("  Run: pip install openai")

try:
    from neo4j import GraphDatabase
    print("✅ Neo4j library loaded")
except ImportError as e:
    print(f"⚠️ Neo4j library not available: {e}")
    print("  Run: pip install neo4j")

try:
    import structlog
    print("✅ Structlog library loaded")
except ImportError as e:
    print(f"⚠️ Structlog library not available: {e}")
    print("  Run: pip install structlog")

print("\n✅ Environment setup complete!")

🔧 Configuration Summary:
  Environment: local
  Config File: .env.local
  LLM Model: gpt-4-turbo
  Neo4j URI: bolt://localhost:7687
  API Host: 0.0.0.0:8000
  Log Level: DEBUG
  Debug Mode: True
  Metrics Enabled: True
  Max Entities: 100
  Entity Confidence Threshold: 0.7
  Fact Confidence Threshold: 0.8

📋 Configuration Messages:
  INFO: Debug mode is enabled
  INFO: Debug logging is enabled

⚠️  Configuration Issues:
  INFO: Debug mode is enabled
  INFO: Debug logging is enabled

✅ Using FastAPI configuration system

🔍 Environment Status:
  Project root: /Users/paulbonneville/Developer/arrgh-fastapi
  Environment: local
  Python path includes src: True

✅ OpenAI API key configured (starts with: sk-svca...)
✅ Neo4j configured and connected: bolt://localhost:7687

📦 Loading dependencies...
✅ HTML processing libraries loaded
✅ OpenAI library loaded
✅ Neo4j library loaded
✅ Structlog library loaded

✅ Environment setup complete!


# Data Models (Pydantic)

**FastAPI File**: `/Users/paulbonneville/Developer/arrgh-fastapi/src/models/newsletter.py`

**Purpose**: Data model definitions using Pydantic for type validation and serialization.

**What this mirrors**: The complete data model system from the FastAPI app, including:
- `Entity` - Represents extracted entities with name, type, confidence, and context
- `Fact` - Represents relationships between entities with temporal context
- `Newsletter` - Core newsletter data structure with HTML content and metadata
- `NewsletterProcessingRequest` - API request model for newsletter processing
- `NewsletterProcessingResponse` - API response model with processing results and metrics
- `ExtractionState` - Internal state management for the processing workflow
- All Pydantic field validators and default factories

---

In [87]:
# Cell 2: Data Models
from typing import Optional, List, Dict, Any
from datetime import datetime

# Import pydantic with fallback
try:
    from pydantic import BaseModel, Field
    print("✅ Pydantic imported successfully")
except ImportError as e:
    print(f"⚠️ Pydantic import failed: {e}")
    print("  Run: pip install pydantic")
    # Create a simple BaseModel fallback
    class BaseModel:
        def __init__(self, **kwargs):
            for key, value in kwargs.items():
                setattr(self, key, value)
    def Field(**kwargs):
        return None

# Data models mirroring src/models/newsletter.py
class Entity(BaseModel):
    """Represents an extracted entity from newsletter content."""
    name: str
    type: str  # Organization, Person, Product, Event, Location, Topic
    aliases: List[str] = Field(default_factory=list)
    confidence: float = Field(ge=0.0, le=1.0)
    context: Optional[str] = None
    properties: Dict[str, Any] = Field(default_factory=dict)

class Fact(BaseModel):
    """Represents a relationship or fact between entities."""
    subject_entity: str
    predicate: str  # relationship type
    object_entity: str
    confidence: float = Field(ge=0.0, le=1.0)
    temporal_context: Optional[str] = None
    date_mentioned: Optional[datetime] = None
    source_context: Optional[str] = None

class Newsletter(BaseModel):
    """Represents a newsletter to be processed."""
    html_content: str
    subject: str
    sender: str
    received_date: Optional[datetime] = None
    newsletter_id: Optional[str] = None

class NewsletterProcessingRequest(BaseModel):
    """Request model for newsletter processing endpoint."""
    html_content: str
    subject: str
    sender: str
    received_date: Optional[datetime] = None

class NewsletterProcessingResponse(BaseModel):
    """Response model for newsletter processing endpoint."""
    status: str
    newsletter_id: str
    processing_time: float
    entities_extracted: int
    entities_new: int
    entities_updated: int
    entity_summary: Dict[str, int]
    text_summary: str
    errors: List[str] = Field(default_factory=list)

class ExtractionState(BaseModel):
    """State for the processing workflow."""
    # Input
    newsletter: Newsletter
    
    # Processing stages
    cleaned_text: str = ""
    extracted_entities: List[Entity] = Field(default_factory=list)
    resolved_entities: List[Entity] = Field(default_factory=list)
    extracted_facts: List[Fact] = Field(default_factory=list)
    
    # Results
    neo4j_updates: Dict[str, Any] = Field(default_factory=dict)
    processing_metrics: Dict[str, Any] = Field(default_factory=dict)
    text_summary: str = ""
    errors: List[str] = Field(default_factory=list)
    
    # Metadata
    processing_start_time: datetime = Field(default_factory=datetime.now)
    current_step: str = "initialized"

print("📊 Data models defined successfully!")
print(f"  - Entity: {Entity.__doc__}")
print(f"  - Newsletter: {Newsletter.__doc__}")
print(f"  - NewsletterProcessingResponse: {NewsletterProcessingResponse.__doc__}")

✅ Pydantic imported successfully
📊 Data models defined successfully!
  - Entity: Represents an extracted entity from newsletter content.
  - Newsletter: Represents a newsletter to be processed.
  - NewsletterProcessingResponse: Response model for newsletter processing endpoint.


# HTML Processor (BeautifulSoup)

**FastAPI File**: `/Users/paulbonneville/Developer/arrgh-fastapi/src/processors/html_processor.py`

**Purpose**: HTML content processing and text extraction for newsletter analysis.

**What this mirrors**: The complete HTML processing system from the FastAPI app, including:
- `HTMLProcessor` class - Main processor for HTML content cleaning and parsing
- `clean_html()` - Removes HTML tags and extracts clean readable text
- `extract_text_sections()` - Structured extraction of headers, paragraphs, links, and lists
- BeautifulSoup parsing with script/style element removal
- Whitespace normalization and text cleaning algorithms
- Structured logging for processing metrics and debugging

---

In [88]:
# Cell 3: HTML Processor (→ )
import structlog

class HTMLProcessor:
    """Process HTML content from newsletters."""
    
    def __init__(self):
        self.logger = structlog.get_logger()
        
    def clean_html(self, html_content: str) -> str:
        """Clean HTML content and extract readable text."""
        try:
            # Parse HTML
            soup = BeautifulSoup(html_content, 'html.parser')
            
            # Remove script and style elements
            for script in soup(["script", "style"]):
                script.decompose()
            
            # Get text
            text = soup.get_text()
            
            # Clean up whitespace
            lines = (line.strip() for line in text.splitlines())
            chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
            text = '\n'.join(chunk for chunk in chunks if chunk)
            
            self.logger.info("HTML content cleaned", 
                           original_length=len(html_content),
                           cleaned_length=len(text))
            
            return text
            
        except Exception as e:
            self.logger.error("Error cleaning HTML", error=str(e))
            return html_content
    
    def extract_text_sections(self, html_content: str) -> Dict[str, Any]:
        """Extract structured text sections from HTML."""
        try:
            soup = BeautifulSoup(html_content, 'html.parser')
            
            # Remove script and style elements
            for script in soup(["script", "style"]):
                script.decompose()
            
            # Extract headers
            headers = []
            for header in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6']):
                text = header.get_text().strip()
                if text:
                    headers.append({
                        'level': header.name,
                        'text': text
                    })
            
            # Extract paragraphs
            paragraphs = []
            for p in soup.find_all('p'):
                text = p.get_text().strip()
                if text and len(text) > 20:  # Filter out short paragraphs
                    paragraphs.append(text)
            
            # Extract links
            links = []
            for link in soup.find_all('a', href=True):
                text = link.get_text().strip()
                href = link['href']
                if text and href:
                    links.append({
                        'text': text,
                        'url': href
                    })
            
            # Extract lists
            lists = []
            for ul in soup.find_all(['ul', 'ol']):
                items = []
                for li in ul.find_all('li'):
                    text = li.get_text().strip()
                    if text:
                        items.append(text)
                if items:
                    lists.append({
                        'type': ul.name,
                        'items': items
                    })
            
            result = {
                'headers': headers,
                'paragraphs': paragraphs,
                'links': links,
                'lists': lists
            }
            
            self.logger.info("Text sections extracted", 
                           headers_count=len(headers),
                           paragraphs_count=len(paragraphs),
                           links_count=len(links),
                           lists_count=len(lists))
            
            return result
            
        except Exception as e:
            self.logger.error("Error extracting text sections", error=str(e))
            return {
                'headers': [],
                'paragraphs': [],
                'links': [],
                'lists': []
            }

# Initialize HTML processor
html_processor = HTMLProcessor()

print("✅ HTML Processor loaded successfully")
print("📄 Functions: clean_html(), extract_text_sections()")

# Test with sample HTML
sample_html = "<html><body><h1>Test</h1><p>This is a test paragraph.</p></body></html>"
test_cleaned = html_processor.clean_html(sample_html)
print(f"🧪 Test: Cleaned {len(sample_html)} chars to {len(test_cleaned)} chars")

✅ HTML Processor loaded successfully
📄 Functions: clean_html(), extract_text_sections()
2025-07-09 21:57:01 [info     ] HTML content cleaned           cleaned_length=29 original_length=71
🧪 Test: Cleaned 71 chars to 29 chars


# Entity Extractor (OpenAI + Pydantic)

**FastAPI File**: `/Users/paulbonneville/Developer/arrgh-fastapi/src/processors/entity_extractor.py`

**Purpose**: Entity extraction using OpenAI LLM with structured JSON output and confidence scoring.

**What this mirrors**: The complete entity extraction system from the FastAPI app, including:
- `EntityExtractor` class - Main class for LLM-based entity extraction
- `ENTITY_EXTRACTION_PROMPT` - Detailed prompt template for structured entity extraction
- `extract_entities()` - Core extraction method with JSON parsing and error handling
- OpenAI client initialization and configuration management
- JSON parsing fixes for production bug (code block removal, brace matching)
- Entity filtering by confidence threshold and validation
- Support for 6 entity types: Organization, Person, Product, Event, Location, Topic
- Structured logging and comprehensive error handling

---

In [89]:
# Cell 4: Entity Extractor (→ ../src/processors/entity_extractor.py)
import json
import re
from typing import List

print("🧪 Testing Cell 4: Entity Extractor")

# ULTRAFIX: Initialize OpenAI client with explicit proxy avoidance
openai_client = None
if config.openai_api_key and not config.openai_api_key.startswith("sk-your-"):
    try:
        # Clear any potential proxy environment variables that might interfere
        import os
        old_env = {}
        proxy_vars = ['HTTP_PROXY', 'HTTPS_PROXY', 'ALL_PROXY', 'http_proxy', 'https_proxy', 'all_proxy']
        for var in proxy_vars:
            if var in os.environ:
                old_env[var] = os.environ[var]
                del os.environ[var]
        
        from openai import OpenAI
        import httpx
        
        # Create httpx client explicitly without proxy to avoid parameter mismatch
        http_client = httpx.Client(
            timeout=30.0,
            proxy=None,  # Explicitly set to None to avoid proxy issues
            trust_env=False  # Don't trust environment variables for proxy config
        )
        
        # Initialize OpenAI client with explicit http_client
        openai_client = OpenAI(
            api_key=config.openai_api_key,
            http_client=http_client
        )
        
        # Restore environment variables
        for var, value in old_env.items():
            os.environ[var] = value
        
        print("✅ OpenAI client initialized successfully with explicit proxy avoidance")
        
        # Test the client with a simple call
        test_response = openai_client.chat.completions.create(
            model=config.llm_model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": "Say 'Hello, I am working!'"}
            ],
            temperature=0.1,
            max_tokens=50
        )
        
        print(f"✅ OpenAI test call successful: {test_response.choices[0].message.content}")
        
    except Exception as e:
        print(f"❌ OpenAI client initialization failed: {e}")
        print(f"  Error type: {type(e).__name__}")
        import traceback
        traceback.print_exc()
        openai_client = None
else:
    print("⚠️ OpenAI API key not configured properly")
    print("  Make sure OPENAI_API_KEY is set in your .env.local file")

# Entity extraction prompt template (mirroring FastAPI exactly)
ENTITY_EXTRACTION_PROMPT = """
You are an expert at extracting structured information from newsletter content. 
Extract entities from the following newsletter text and classify them into these categories:

**Entity Types:**
- **Organization**: Companies, institutions, government bodies
- **Person**: Individuals mentioned in content
- **Product**: Software, hardware, services, models
- **Event**: Conferences, announcements, launches
- **Location**: Geographic locations (cities, countries, regions)
- **Topic**: Subject areas, technologies, fields of study

**Instructions:**
1. Extract entities with high confidence (>0.7)
2. Provide alternative names/aliases if mentioned
3. Include context where the entity was mentioned
4. Rate confidence from 0.0 to 1.0
5. Return results as valid JSON

**Newsletter Content:**
{content}

**Required JSON Format:**
```json
{{
  "entities": [
    {{
      "name": "Entity Name",
      "type": "Organization|Person|Product|Event|Location|Topic",
      "aliases": ["Alternative Name 1", "Alternative Name 2"],
      "confidence": 0.95,
      "context": "The sentence or phrase where this entity was mentioned",
      "properties": {{
        "additional_info": "any relevant details"
      }}
    }}
  ]
}}
```

Return only valid JSON, no additional text.
"""

def extract_entities_from_text(content: str) -> List[Entity]:
    """
    Extract entities from content using OpenAI.
    
    This function mirrors the logic in src/processors/entity_extractor.py
    but includes the JSON parsing fix for the production bug.
    """
    if not openai_client:
        print("⚠️ OpenAI client not available - check API key configuration")
        return []
    
    if not content.strip():
        print("⚠️ Empty content provided")
        return []
    
    try:
        # Create the completion request (matching FastAPI parameters exactly)
        response = openai_client.chat.completions.create(
            model=config.llm_model,
            messages=[
                {"role": "system", "content": "You are an expert entity extraction assistant."},
                {"role": "user", "content": ENTITY_EXTRACTION_PROMPT.format(content=content[:3000])}
            ],
            temperature=config.llm_temperature,
            max_tokens=config.llm_max_tokens
        )
        
        # Get the response content
        result_text = response.choices[0].message.content.strip()
        
        # Handle JSON parsing with the production bug fix
        # Remove code block markers if present
        if '```json' in result_text:
            json_match = re.search(r'```json\s*(.*?)\s*```', result_text, re.DOTALL)
            if json_match:
                result_text = json_match.group(1).strip()
        elif '```' in result_text:
            json_match = re.search(r'```\s*(.*?)\s*```', result_text, re.DOTALL)
            if json_match:
                result_text = json_match.group(1).strip()
        
        # Parse JSON with enhanced error handling (matching FastAPI fix)
        try:
            result = json.loads(result_text)
        except json.JSONDecodeError as e:
            print(f"⚠️ JSON parsing failed: {e}")
            # Try to find JSON object in the response
            if not result_text.startswith('{'):
                start_idx = result_text.find('{')
                if start_idx != -1:
                    # Find the matching closing brace
                    brace_count = 0
                    end_idx = start_idx
                    for i, char in enumerate(result_text[start_idx:], start_idx):
                        if char == '{':
                            brace_count += 1
                        elif char == '}':
                            brace_count -= 1
                            if brace_count == 0:
                                end_idx = i + 1
                                break
                    
                    if end_idx > start_idx:
                        result_text = result_text[start_idx:end_idx]
                        result = json.loads(result_text)
                    else:
                        raise e
                else:
                    raise e
            else:
                raise e
        
        # Convert to Entity objects
        entities = []
        for entity_data in result.get('entities', []):
            # Validate required fields
            if not entity_data.get('name') or not entity_data.get('type'):
                continue
            
            # Filter by confidence threshold
            confidence = entity_data.get('confidence', 0.0)
            if confidence < config.entity_confidence_threshold:
                continue
            
            entity = Entity(
                name=entity_data['name'],
                type=entity_data['type'],
                aliases=entity_data.get('aliases', []),
                confidence=confidence,
                context=entity_data.get('context'),
                properties=entity_data.get('properties', {})
            )
            entities.append(entity)
        
        return entities
    
    except Exception as e:
        print(f"❌ Entity extraction failed: {e}")
        return []

# Test function
def test_entity_extraction():
    """Test the entity extraction function."""
    test_content = """
    OpenAI announced GPT-4 at their developer conference in San Francisco. 
    CEO Sam Altman presented the new capabilities during the OpenAI DevDay 2024 event.
    Microsoft expanded their Azure AI services with enterprise features.
    """
    
    print("🔍 Testing entity extraction...")
    entities = extract_entities_from_text(test_content)
    
    if entities:
        print(f"✅ Extracted {len(entities)} entities:")
        for i, entity in enumerate(entities[:5]):  # Show first 5
            print(f"  {i+1}. {entity.name} ({entity.type}) - Confidence: {entity.confidence}")
    else:
        print("⚠️ No entities extracted")
    
    return entities

# Run test if OpenAI is available
if openai_client:
    test_entities = test_entity_extraction()
else:
    print("⚠️ Skipping entity extraction test - OpenAI client not available")
    test_entities = []

print("\n✅ Entity extraction setup complete!")

🧪 Testing Cell 4: Entity Extractor
✅ OpenAI client initialized successfully with explicit proxy avoidance
✅ OpenAI test call successful: Hello, I am working!
🔍 Testing entity extraction...
✅ Extracted 7 entities:
  1. OpenAI (Organization) - Confidence: 0.99
  2. GPT-4 (Product) - Confidence: 0.98
  3. San Francisco (Location) - Confidence: 0.95
  4. Sam Altman (Person) - Confidence: 0.97
  5. OpenAI DevDay 2024 (Event) - Confidence: 0.95

✅ Entity extraction setup complete!


In [90]:
# Cell 5: Neo4j Client (→ ../src/graph/neo4j_client.py)
import uuid
from typing import Dict, List, Optional, Any
from datetime import datetime

class Neo4jClient:
    """Neo4j client for graph database operations."""
    
    def __init__(self, uri: str = None, user: str = None, password: str = None):
        """Initialize Neo4j client with configuration."""
        self.uri = uri or config.neo4j_uri
        self.user = user or config.neo4j_user
        self.password = password or config.neo4j_password
        self.driver = None
        self.connected = False
        
    def connect(self) -> bool:
        """Establish connection to Neo4j database."""
        try:
            self.driver = GraphDatabase.driver(
                self.uri,
                auth=(self.user, self.password)
            )
            
            # Test connection
            with self.driver.session() as session:
                result = session.run("RETURN 1 as test")
                result.single()
            
            self.connected = True
            print("✅ Neo4j connection established")
            return True
            
        except Exception as e:
            print(f"❌ Neo4j connection failed: {e}")
            print("  Check if Neo4j is running and credentials are correct")
            return False
    
    def disconnect(self):
        """Close Neo4j connection."""
        if self.driver:
            self.driver.close()
            self.connected = False
            print("🔌 Neo4j connection closed")
    
    def execute_query(self, query: str, parameters: Dict[str, Any] = None) -> List[Dict]:
        """Execute a Cypher query and return results."""
        if not self.connected or not self.driver:
            print("❌ No active Neo4j connection")
            return []
        
        try:
            with self.driver.session() as session:
                result = session.run(query, parameters or {})
                return result.data()
        except Exception as e:
            print(f"❌ Query execution failed: {e}")
            return []
    
    def create_or_update_entity(self, entity: Entity) -> Optional[Dict]:
        """Create or update an entity node in the graph."""
        # Convert properties to JSON string to avoid Neo4j type issues
        properties_json = None
        if entity.properties:
            import json
            properties_json = json.dumps(entity.properties)
        
        query = f"""
        MERGE (e:{entity.type} {{name: $name}})
        ON CREATE SET 
            e.created_at = datetime(),
            e.confidence = $confidence,
            e.aliases = $aliases,
            e.mention_count = 1,
            e.last_seen = datetime(),
            e.properties_json = $properties_json
        ON MATCH SET
            e.last_seen = datetime(),
            e.mention_count = e.mention_count + 1,
            e.confidence = CASE 
                WHEN $confidence > e.confidence THEN $confidence 
                ELSE e.confidence 
            END,
            e.properties_json = COALESCE($properties_json, e.properties_json)
        RETURN e, 
               CASE WHEN e.created_at = e.last_seen THEN 'created' ELSE 'updated' END as operation
        """
        
        parameters = {
            'name': entity.name,
            'confidence': entity.confidence,
            'aliases': entity.aliases,
            'properties_json': properties_json
        }
        
        result = self.execute_query(query, parameters)
        return result[0] if result else None
    
    def create_newsletter_node(self, newsletter: Newsletter) -> Optional[Dict]:
        """Create a newsletter node in the graph."""
        query = """
        MERGE (n:Newsletter {id: $newsletter_id})
        ON CREATE SET
            n.subject = $subject,
            n.sender = $sender,
            n.received_date = $received_date,
            n.created_at = datetime(),
            n.content_length = $content_length,
            n.processed = true
        ON MATCH SET
            n.last_processed = datetime(),
            n.processed = true
        RETURN n
        """
        
        parameters = {
            'newsletter_id': newsletter.newsletter_id,
            'subject': newsletter.subject,
            'sender': newsletter.sender,
            'received_date': newsletter.received_date.isoformat() if newsletter.received_date else None,
            'content_length': len(newsletter.html_content)
        }
        
        result = self.execute_query(query, parameters)
        return result[0] if result else None
    
    def link_entity_to_newsletter(self, entity_name: str, entity_type: str, newsletter_id: str, context: str = None) -> Optional[Dict]:
        """Create a MENTIONED_IN relationship between entity and newsletter."""
        query = f"""
        MATCH (e:{entity_type} {{name: $entity_name}})
        MATCH (n:Newsletter {{id: $newsletter_id}})
        MERGE (e)-[r:MENTIONED_IN]->(n)
        ON CREATE SET
            r.created_at = datetime(),
            r.context = $context,
            r.mention_count = 1
        ON MATCH SET
            r.last_mentioned = datetime(),
            r.mention_count = r.mention_count + 1,
            r.context = COALESCE($context, r.context)
        RETURN r
        """
        
        parameters = {
            'entity_name': entity_name,
            'newsletter_id': newsletter_id,
            'context': context
        }
        
        result = self.execute_query(query, parameters)
        return result[0] if result else None
    
    def get_graph_stats(self) -> Dict[str, int]:
        """Get statistics about the graph database."""
        query = """
        CALL {
            MATCH (o:Organization) RETURN count(o) as organizations
        }
        CALL {
            MATCH (p:Person) RETURN count(p) as people
        }
        CALL {
            MATCH (pr:Product) RETURN count(pr) as products
        }
        CALL {
            MATCH (e:Event) RETURN count(e) as events
        }
        CALL {
            MATCH (l:Location) RETURN count(l) as locations
        }
        CALL {
            MATCH (t:Topic) RETURN count(t) as topics
        }
        CALL {
            MATCH (n:Newsletter) RETURN count(n) as newsletters
        }
        CALL {
            MATCH ()-[r]->() RETURN count(r) as relationships
        }
        RETURN organizations, people, products, events, locations, topics, newsletters, relationships
        """
        
        result = self.execute_query(query)
        return result[0] if result else {}

# Initialize Neo4j client
neo4j_client = Neo4jClient()

# Test connection
if neo4j_client.connect():
    # Display current graph statistics
    stats = neo4j_client.get_graph_stats()
    print("\n📊 Current graph statistics:")
    for key, value in stats.items():
        print(f"  {key.capitalize()}: {value}")
    
    # Test basic query
    test_result = neo4j_client.execute_query("RETURN 'Neo4j is working!' as message")
    if test_result:
        print(f"✅ Test query result: {test_result[0]['message']}")
else:
    print("⚠️ Neo4j operations will be limited without connection")

print("\n✅ Neo4j client setup complete!")

✅ Neo4j connection established

📊 Current graph statistics:
  Organizations: 6
  People: 3
  Products: 3
  Events: 3
  Locations: 2
  Topics: 2
  Newsletters: 11
  Relationships: 29
✅ Test query result: Neo4j is working!

✅ Neo4j client setup complete!


# Newsletter Processor (Complete Workflow)

**FastAPI File**: `/Users/paulbonneville/Developer/arrgh-fastapi/src/workflows/newsletter_processor.py`

**Purpose**: Complete newsletter processing pipeline orchestrating all components for end-to-end workflow.

**What this mirrors**: The complete workflow orchestration from the FastAPI app, including:
- `NewsletterProcessor` class - Main orchestrator for the complete processing pipeline
- `process_newsletter()` - 5-step workflow: HTML cleaning → entity extraction → Neo4j storage → summary generation
- `_generate_text_summary()` - Intelligent summary creation with entity breakdown and metrics
- Component integration: HTMLProcessor, EntityExtractor, Neo4jClient coordination
- Neo4j graph operations: entity creation/updates, relationship linking, newsletter node management
- Performance metrics and processing time tracking
- Comprehensive error handling with rollback capabilities
- Response generation with detailed processing statistics

---

In [91]:
import uuid
from datetime import datetime
from typing import Dict, List, Any

class NewsletterProcessor:
    """Complete newsletter processing pipeline."""
    
    def __init__(self, config_obj):
        self.config = config_obj
        self.html_processor = html_processor
        self.openai_client = openai_client
        self.neo4j_client = neo4j_client
        print("✅ Newsletter processor initialized")
    
    def process_newsletter(self, newsletter: Newsletter) -> NewsletterProcessingResponse:
        """Process a newsletter through the complete pipeline."""
        start_time = datetime.now()
        newsletter_id = newsletter.newsletter_id or str(uuid.uuid4())
        newsletter.newsletter_id = newsletter_id
        
        print(f"🚀 Starting newsletter processing: {newsletter.subject}")
        
        try:
            # Step 1: Clean HTML content
            print("1️⃣ Processing HTML content...")
            cleaned_text = self.html_processor.clean_html(newsletter.html_content)
            text_sections = self.html_processor.extract_text_sections(newsletter.html_content)
            print(f"   ✅ Cleaned text: {len(cleaned_text)} characters")
            
            # Step 2: Extract entities
            print("2️⃣ Extracting entities...")
            entities = extract_entities_from_text(cleaned_text)
            print(f"   ✅ Extracted {len(entities)} entities")
            
            # Step 3: Create newsletter node in Neo4j
            entities_new = 0
            entities_updated = 0
            relationships_created = 0
            
            if self.neo4j_client.connected:
                print("3️⃣ Creating newsletter node...")
                newsletter_result = self.neo4j_client.create_newsletter_node(newsletter)
                if newsletter_result:
                    print("   ✅ Newsletter node created")
                
                # Step 4: Process entities in Neo4j
                print("4️⃣ Processing entities in graph...")
                for entity in entities:
                    try:
                        # Create or update entity
                        entity_result = self.neo4j_client.create_or_update_entity(entity)
                        if entity_result:
                            operation = entity_result.get('operation', 'unknown')
                            if operation == 'created':
                                entities_new += 1
                            else:
                                entities_updated += 1
                            
                            # Link to newsletter
                            link_result = self.neo4j_client.link_entity_to_newsletter(
                                entity.name,
                                entity.type,
                                newsletter.newsletter_id,
                                entity.context
                            )
                            if link_result:
                                relationships_created += 1
                    except Exception as e:
                        print(f"   ⚠️ Error processing entity {entity.name}: {e}")
                
                print(f"   ✅ Processed: {entities_new} new, {entities_updated} updated, {relationships_created} linked")
            
            # Step 5: Generate summary
            print("5️⃣ Generating summary...")
            processing_time = (datetime.now() - start_time).total_seconds()
            
            # Entity type breakdown
            entity_summary = {}
            for entity in entities:
                entity_summary[entity.type] = entity_summary.get(entity.type, 0) + 1
            
            # Generate text summary
            text_summary = self._generate_text_summary(cleaned_text, entities, text_sections)
            
            print(f"   ✅ Processing completed in {processing_time:.2f} seconds")
            
            return NewsletterProcessingResponse(
                status="success",
                newsletter_id=newsletter_id,
                processing_time=processing_time,
                entities_extracted=len(entities),
                entities_new=entities_new,
                entities_updated=entities_updated,
                entity_summary=entity_summary,
                text_summary=text_summary,
                errors=[]
            )
            
        except Exception as e:
            processing_time = (datetime.now() - start_time).total_seconds()
            error_msg = f"Error processing newsletter: {str(e)}"
            print(f"❌ Pipeline failed: {error_msg}")
            
            return NewsletterProcessingResponse(
                status="error",
                newsletter_id=newsletter_id,
                processing_time=processing_time,
                entities_extracted=0,
                entities_new=0,
                entities_updated=0,
                entity_summary={},
                text_summary="",
                errors=[error_msg]
            )
    
    def _generate_text_summary(self, cleaned_text: str, entities: List[Entity], text_sections: Dict[str, Any]) -> str:
        """Generate a text summary of the newsletter."""
        summary_parts = []
        
        # Add basic stats
        summary_parts.append(f"Newsletter processed with {len(entities)} entities extracted.")
        
        # Add entity type breakdown
        entity_types = {}
        for entity in entities:
            entity_types[entity.type] = entity_types.get(entity.type, 0) + 1
        
        if entity_types:
            type_summary = ", ".join([f"{count} {entity_type}" for entity_type, count in entity_types.items()])
            summary_parts.append(f"Entity breakdown: {type_summary}.")
        
        # Add content structure info
        if text_sections:
            structure_info = []
            if text_sections.get('headers'):
                structure_info.append(f"{len(text_sections['headers'])} headers")
            if text_sections.get('paragraphs'):
                structure_info.append(f"{len(text_sections['paragraphs'])} paragraphs")
            if text_sections.get('links'):
                structure_info.append(f"{len(text_sections['links'])} links")
            
            if structure_info:
                summary_parts.append(f"Content structure: {', '.join(structure_info)}.")
        
        # Add top entities
        if entities:
            top_entities = sorted(entities, key=lambda e: e.confidence, reverse=True)[:5]
            entity_names = [e.name for e in top_entities]
            summary_parts.append(f"Top entities: {', '.join(entity_names)}.")
        
        return " ".join(summary_parts)

# Initialize newsletter processor
newsletter_processor = NewsletterProcessor(config)

print("✅ Newsletter Processor loaded successfully")
print("🔄 Complete pipeline: HTML → Entities → Neo4j → Summary")
print("📊 Functions: process_newsletter()")

✅ Newsletter processor initialized
✅ Newsletter Processor loaded successfully
🔄 Complete pipeline: HTML → Entities → Neo4j → Summary
📊 Functions: process_newsletter()


# Testing & Validation (Complete Pipeline Test)

**FastAPI File**: Testing framework (not directly mirrored - notebook-specific utilities)

**Purpose**: Complete pipeline testing and validation with comprehensive test scenarios and performance assessment.

**What this mirrors**: Testing utilities that validate the complete FastAPI workflow, including:
- `run_complete_pipeline_test()` - Full end-to-end pipeline test with realistic AI newsletter content
- `validate_pipeline_results()` - Performance and quality assessment with multiple metrics
- `quick_test()` - Minimal test for rapid debugging and component validation
- Sample data generation with rich entity content (companies, people, products, events, locations)
- Performance benchmarking: processing time, entity extraction quality, success rates
- Error handling validation and pipeline robustness testing
- Neo4j integration testing with graph database operations
- Comprehensive reporting with success/failure analysis and recommendations

---

In [92]:
from datetime import datetime, timezone
import time

def run_complete_pipeline_test():
    """Test the complete newsletter processing pipeline."""
    
    # Sample newsletter for testing
    sample_html = """
    <!DOCTYPE html>
    <html>
    <head><title>AI Weekly Newsletter #245</title></head>
    <body>
        <h1>AI Weekly Newsletter #245</h1>
        <p>Welcome to this week's AI updates!</p>
        
        <h2>Major Announcements</h2>
        <p><strong>OpenAI</strong> announced the release of <strong>GPT-5</strong> at their 
        developer conference in <strong>San Francisco</strong>. CEO <strong>Sam Altman</strong> 
        presented the new capabilities during the <strong>OpenAI DevDay 2024</strong> event.</p>
        
        <h2>Company Updates</h2>
        <p><strong>Microsoft</strong> expanded their <strong>Azure AI</strong> services with 
        new enterprise features. The announcement was made by <strong>Satya Nadella</strong> 
        during the <strong>Microsoft Build 2024</strong> conference.</p>
        
        <h2>Industry News</h2>
        <p><strong>Google</strong> launched their new <strong>Gemini Pro</strong> model, 
        focusing on <strong>AI Safety</strong> and <strong>Responsible AI</strong> development. 
        The launch event was held at <strong>Google I/O 2024</strong> in <strong>Mountain View</strong>.</p>
        
        <h2>Educational Content</h2>
        <p><strong>Stanford University</strong> announced a new <strong>AI Safety</strong> course 
        taught by renowned professor <strong>Fei-Fei Li</strong>. The course will cover 
        <strong>Machine Learning</strong> ethics and <strong>AI Alignment</strong>.</p>
        
        <h2>Research Highlights</h2>
        <p>New research on <strong>Quantum Computing</strong> applications in <strong>AI</strong> 
        was published by researchers at <strong>MIT</strong> and <strong>IBM Research</strong>. 
        The paper explores <strong>Quantum Machine Learning</strong> algorithms.</p>
        
        <p>Thanks for reading! See you next week.</p>
        <p>Best regards,<br>The AI Weekly Team</p>
    </body>
    </html>
    """
    
    # Create newsletter object
    newsletter = Newsletter(
        html_content=sample_html,
        subject="AI Weekly Newsletter #245 - Development Test",
        sender="ai-weekly@example.com",
        received_date=datetime.now(timezone.utc),
        newsletter_id=f"ai-weekly-245-test-{int(time.time())}"
    )
    
    print("🚀 Starting complete pipeline test...")
    print(f"  Newsletter: {newsletter.subject}")
    print(f"  Newsletter ID: {newsletter.newsletter_id}")
    
    # Use the newsletter processor
    try:
        response = newsletter_processor.process_newsletter(newsletter)
        
        print(f"\n✅ Pipeline completed: {response.status}")
        print(f"  - Processing time: {response.processing_time:.2f}s")
        print(f"  - Entities extracted: {response.entities_extracted}")
        print(f"  - New entities: {response.entities_new}")
        print(f"  - Updated entities: {response.entities_updated}")
        print(f"  - Entity summary: {response.entity_summary}")
        print(f"  - Text summary: {response.text_summary}")
        
        if response.errors:
            print(f"  - Errors: {response.errors}")
            
        return response
        
    except Exception as e:
        print(f"❌ Pipeline test failed: {e}")
        import traceback
        traceback.print_exc()
        return None

def validate_pipeline_results(response) -> bool:
    """Validate pipeline results and provide feedback."""
    if not response:
        print("❌ No response to validate")
        return False
    
    print("\n📋 Pipeline Validation Results:")
    print(f"  Status: {response.status}")
    print(f"  Processing time: {response.processing_time:.2f} seconds")
    print(f"  Entities extracted: {response.entities_extracted}")
    
    if neo4j_client.connected:
        print(f"  Entities new: {response.entities_new}")
        print(f"  Entities updated: {response.entities_updated}")
    
    if response.errors:
        print(f"  ❌ Errors ({len(response.errors)}):")
        for error in response.errors:
            print(f"    - {error}")
    
    # Performance assessment
    if response.processing_time < 10:
        print("  ⚡ Performance: Excellent (< 10s)")
    elif response.processing_time < 30:
        print("  ✅ Performance: Good (< 30s)")
    else:
        print("  ⚠️ Performance: Slow (> 30s)")
    
    # Entity extraction assessment
    if response.entities_extracted > 15:
        print("  🎯 Entity extraction: Excellent (> 15 entities)")
    elif response.entities_extracted > 5:
        print("  ✅ Entity extraction: Good (> 5 entities)")
    elif response.entities_extracted > 0:
        print("  ⚠️ Entity extraction: Limited (1-5 entities)")
    else:
        print("  ❌ Entity extraction: Failed (0 entities)")
    
    return response.status == 'success' and len(response.errors) == 0

def quick_test():
    """Quick test with minimal content."""
    test_html = "<html><body><h1>Test</h1><p>OpenAI released GPT-4 in San Francisco.</p></body></html>"
    newsletter = Newsletter(
        html_content=test_html,
        subject="Quick Test",
        sender="test@example.com",
        newsletter_id=f"quick-test-{int(time.time())}"
    )
    
    print("🔬 Running quick test...")
    response = newsletter_processor.process_newsletter(newsletter)
    print(f"Quick test result: {response.status}, {response.entities_extracted} entities")
    return response

# Run the complete pipeline test
print("🔬 Running complete pipeline test...")
test_response = run_complete_pipeline_test()

# Validate results
print("\n" + "="*50)
validation_passed = validate_pipeline_results(test_response)

if validation_passed:
    print("\n🎉 All tests passed! Pipeline is working correctly.")
else:
    print("\n⚠️ Some tests failed. Check the results above.")
    print("\n💡 Running quick test to diagnose...")
    quick_response = quick_test()

print("\n✅ Pipeline testing complete!")

🔬 Running complete pipeline test...
🚀 Starting complete pipeline test...
  Newsletter: AI Weekly Newsletter #245 - Development Test
  Newsletter ID: ai-weekly-245-test-1752119831
🚀 Starting newsletter processing: AI Weekly Newsletter #245 - Development Test
1️⃣ Processing HTML content...
2025-07-09 21:57:11 [info     ] HTML content cleaned           cleaned_length=1058 original_length=1909
2025-07-09 21:57:11 [info     ] Text sections extracted        headers_count=6 links_count=0 lists_count=0 paragraphs_count=8
   ✅ Cleaned text: 1058 characters
2️⃣ Extracting entities...
   ✅ Extracted 19 entities
3️⃣ Creating newsletter node...
   ✅ Newsletter node created
4️⃣ Processing entities in graph...
   ✅ Processed: 0 new, 19 updated, 19 linked
5️⃣ Generating summary...
   ✅ Processing completed in 20.53 seconds

✅ Pipeline completed: success
  - Processing time: 20.53s
  - Entities extracted: 19
  - New entities: 0
  - Updated entities: 19
  - Entity summary: {'Organization': 6, 'Product':

# Code Export Helper (Sync changes back to FastAPI)

**FastAPI File**: Development utilities (not directly mirrored - notebook-specific development tools)

**Purpose**: Synchronization tools for maintaining code consistency between notebook development and FastAPI production files.

**What this mirrors**: Development workflow utilities that facilitate notebook-to-FastAPI synchronization, including:
- `compare_files()` - File difference analysis between notebook code and FastAPI files
- `export_to_fastapi()` - Safe code export with automatic backup creation
- `sync_status_check()` - Comprehensive status check for all components across notebook and FastAPI
- `validate_fastapi_imports()` - Import validation to ensure exported code works in FastAPI context
- `development_summary()` - Environment status dashboard with configuration and test results
- `quick_development_check()` - Complete development environment validation workflow
- File mapping system for all FastAPI components (config, models, processors, workflows)
- Development lifecycle management with backup and rollback capabilities

---

In [93]:
import difflib
from pathlib import Path
import shutil

# Define file mappings
FILE_MAPPINGS = {
    'config': '/Users/paulbonneville/Developer/arrgh-fastapi/src/config.py',
    'models': '/Users/paulbonneville/Developer/arrgh-fastapi/src/models/newsletter.py',
    'html_processor': '/Users/paulbonneville/Developer/arrgh-fastapi/src/processors/html_processor.py',
    'entity_extractor': '/Users/paulbonneville/Developer/arrgh-fastapi/src/processors/entity_extractor.py',
    'neo4j_client': '/Users/paulbonneville/Developer/arrgh-fastapi/src/graph/neo4j_client.py',
    'newsletter_processor': '/Users/paulbonneville/Developer/arrgh-fastapi/src/workflows/newsletter_processor.py'
}

def compare_files(notebook_code: str, fastapi_file: str) -> bool:
    """Compare notebook code with FastAPI file."""
    try:
        with open(fastapi_file, 'r') as f:
            fastapi_code = f.read()
        
        # Simple comparison - in production, you'd want more sophisticated matching
        notebook_lines = notebook_code.strip().split('\n')
        fastapi_lines = fastapi_code.strip().split('\n')
        
        # Show diff if different
        if notebook_lines != fastapi_lines:
            print(f"📊 Differences found in {Path(fastapi_file).name}:")
            diff = difflib.unified_diff(
                fastapi_lines, notebook_lines,
                fromfile='FastAPI', tofile='Notebook',
                lineterm='', n=3
            )
            for line in list(diff)[:20]:  # Show first 20 lines of diff
                print(line)
            return False
        else:
            print(f"✅ {Path(fastapi_file).name} matches notebook code")
            return True
            
    except FileNotFoundError:
        print(f"❌ FastAPI file not found: {fastapi_file}")
        return False
    except Exception as e:
        print(f"❌ Error comparing files: {str(e)}")
        return False

def export_to_fastapi(component: str, notebook_code: str, backup: bool = True) -> bool:
    """Export notebook code to FastAPI file."""
    if component not in FILE_MAPPINGS:
        print(f"❌ Unknown component: {component}")
        return False
    
    fastapi_file = FILE_MAPPINGS[component]
    
    try:
        # Create backup if requested
        if backup and Path(fastapi_file).exists():
            backup_file = f"{fastapi_file}.backup"
            shutil.copy2(fastapi_file, backup_file)
            print(f"📋 Backup created: {backup_file}")
        
        # Write notebook code to FastAPI file
        with open(fastapi_file, 'w') as f:
            f.write(notebook_code)
        
        print(f"✅ Exported to {Path(fastapi_file).name}")
        return True
        
    except Exception as e:
        print(f"❌ Export failed: {str(e)}")
        return False

def sync_status_check():
    """Check sync status of all components."""
    print("🔄 Checking sync status with FastAPI...\n")
    
    sync_status = {}
    
    # Note: In a real implementation, you'd extract the actual code from notebook cells
    # This is a placeholder showing the concept
    
    components = ['html_processor', 'entity_extractor', 'neo4j_client', 'newsletter_processor']
    
    for component in components:
        print(f"📦 Checking {component}...")
        
        # This would compare actual notebook cell code with FastAPI files
        # For now, we'll just check if files exist
        fastapi_file = FILE_MAPPINGS[component]
        if Path(fastapi_file).exists():
            sync_status[component] = "✅ File exists"
        else:
            sync_status[component] = "❌ File missing"
        
        print(f"  {sync_status[component]}")
        print()
    
    # Summary
    print("📊 Sync Status Summary:")
    for component, status in sync_status.items():
        print(f"  - {component}: {status}")
    
    return sync_status

def validate_fastapi_imports():
    """Validate that FastAPI can import our changes."""
    print("🔍 Validating FastAPI imports...")
    
    try:
        # Test basic imports
        import config
        print("✅ config.py imports successfully")
        
        from models import newsletter
        print("✅ models/newsletter.py imports successfully")
        
        # Test processor imports
        try:
            from processors import html_processor
            print("✅ processors/html_processor.py imports successfully")
        except ImportError as e:
            print(f"⚠️ html_processor import issue: {e}")
        
        try:
            from processors import entity_extractor
            print("✅ processors/entity_extractor.py imports successfully")
        except ImportError as e:
            print(f"⚠️ entity_extractor import issue: {e}")
        
        return True
        
    except Exception as e:
        print(f"❌ Import validation failed: {str(e)}")
        return False

def development_summary():
    """Provide a summary of the development environment."""
    print("📋 Development Environment Summary")
    print("="*40)
    
    # Configuration status
    print(f"✅ OpenAI configured: {bool(config.openai_api_key and not config.openai_api_key.startswith('sk-your-'))}")
    print(f"✅ Neo4j connected: {neo4j_client.connected if 'neo4j_client' in globals() else False}")
    print(f"✅ HTML Processor ready: {'html_processor' in globals()}")
    print(f"✅ Entity Extractor ready: {'openai_client' in globals() and openai_client is not None}")
    print(f"✅ Newsletter Processor ready: {'newsletter_processor' in globals()}")
    
    # Recent test results
    if 'test_response' in globals() and test_response:
        print(f"\n📊 Last Test Results:")
        print(f"  Status: {test_response.status}")
        print(f"  Entities: {test_response.entities_extracted}")
        print(f"  Time: {test_response.processing_time:.2f}s")
    
    print(f"\n🎯 Ready for development and testing!")

# Helper functions for development workflow
def quick_development_check():
    """Quick check of development status."""
    print("🚀 Quick Development Status Check\n")
    
    # Check sync status
    print("1. Checking sync status...")
    sync_status = sync_status_check()
    
    # Validate components
    print("\n2. Validating components...")
    validation_results = validate_fastapi_imports()
    
    # Development summary
    print("\n3. Development summary...")
    development_summary()
    
    print("\n🎉 Development check completed!")
    return {
        'sync_status': sync_status,
        'validation_results': validation_results
    }

def extract_cell_code(cell_number: int) -> str:
    """Extract code from a specific notebook cell."""
    # This is a placeholder - in a real implementation, you'd parse the notebook JSON
    print(f"📄 Extracting code from cell {cell_number}...")
    print("⚠️  Manual extraction required - copy code from notebook cell")
    return ""

print("✅ Code Export Helper loaded successfully")
print("🔄 Functions: compare_files(), export_to_fastapi(), sync_status_check()")
print("📁 File mappings configured for all components")
print("\n💡 Usage:")
print("  - sync_status_check(): Check sync status of all components")
print("  - quick_development_check(): Run development status check")
print("  - validate_fastapi_imports(): Test FastAPI imports")
print("  - development_summary(): Show current environment status")

# Run quick check
result = quick_development_check()

✅ Code Export Helper loaded successfully
🔄 Functions: compare_files(), export_to_fastapi(), sync_status_check()
📁 File mappings configured for all components

💡 Usage:
  - sync_status_check(): Check sync status of all components
  - quick_development_check(): Run development status check
  - validate_fastapi_imports(): Test FastAPI imports
  - development_summary(): Show current environment status
🚀 Quick Development Status Check

1. Checking sync status...
🔄 Checking sync status with FastAPI...

📦 Checking html_processor...
  ✅ File exists

📦 Checking entity_extractor...
  ✅ File exists

📦 Checking neo4j_client...
  ✅ File exists

📦 Checking newsletter_processor...
  ✅ File exists

📊 Sync Status Summary:
  - html_processor: ✅ File exists
  - entity_extractor: ✅ File exists
  - neo4j_client: ✅ File exists
  - newsletter_processor: ✅ File exists

2. Validating components...
🔍 Validating FastAPI imports...
✅ config.py imports successfully
✅ models/newsletter.py imports successfully
✅ pro

# Development Workflow Summary

This notebook provides a complete 1:1 correspondence with the FastAPI newsletter processing system:

### **Implemented Components**:
1. **Environment Setup** → `src/config.py`
2. **Data Models** → `src/models/newsletter.py`
3. **HTML Processor** → `src/processors/html_processor.py`
4. **Entity Extractor** → `src/processors/entity_extractor.py`
5. **Neo4j Client** → `src/graph/neo4j_client.py`
6. **Newsletter Processor** → `src/workflows/newsletter_processor.py`
7. **Testing Framework** → Complete pipeline validation
8. **Code Export Helper** → Sync changes back to FastAPI

### **Development Workflow**:
1. **Edit** code in notebook cells with immediate feedback
2. **Test** changes using the testing framework
3. **Validate** with Neo4j browser and sample data
4. **Export** working code to FastAPI files
5. **Deploy** tested changes to production

### **Next Steps**:
- Run `run_all_tests()` to validate the complete pipeline
- Use `quick_test_and_sync()` for rapid development cycles
- Test with your own newsletter data
- Export working changes to FastAPI using the sync functions

### **Key Features**:
- **Immediate Feedback**: Test changes instantly in notebook cells
- **Complete Pipeline**: Full newsletter processing from HTML to Neo4j
- **Robust Error Handling**: Matches production FastAPI implementation
- **Easy Synchronization**: Export tested code back to FastAPI files
- **Interactive Development**: Neo4j browser integration for graph exploration

Happy coding!